In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [2]:
training_data = pd.read_pickle('../data/training_data.pkl')

In [3]:
# Get Labels
Y = pd.get_dummies(training_data.call).astype(float).values
# Get training data as numpy array
X = training_data.drop('call', axis=1).astype(float).values

In [5]:
X.shape

(37072, 67)

In [6]:
Y.shape

(37072, 4)

In [7]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [8]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(67, input_dim=67, kernel_initializer='normal', activation='relu'))
    model.add(Dense(23, activation='relu'))
    model.add(Dense(23, activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=5, verbose=1)

In [10]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [11]:
results = cross_val_score(estimator, X, Y, cv=kfold)
print("\nBaseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
33364/33364 [==============================] - 35s - loss: 0.7362 - acc: 0.6865    
Epoch 2/10
33364/33364 [==============================] - 35s - loss: 0.6114 - acc: 0.7523    
Epoch 3/10
33364/33364 [==============================] - 31s - loss: 0.5786 - acc: 0.7671    
Epoch 4/10
33364/33364 [==============================] - 32s - loss: 0.5620 - acc: 0.7729    
Epoch 5/10
33364/33364 [==============================] - 34s - loss: 0.5501 - acc: 0.7778    
Epoch 6/10
33364/33364 [==============================] - 32s - loss: 0.5406 - acc: 0.7820    
Epoch 7/10
33364/33364 [==============================] - 32s - loss: 0.5325 - acc: 0.7857    
Epoch 8/10
33364/33364 [==============================] - 32s - loss: 0.5272 - acc: 0.7873    
Epoch 9/10
33364/33364 [==============================] - 31s - loss: 0.5218 - acc: 0.7889    
Epoch 10/10
33364/33364 [==============================] - 35s - loss: 0.7351 - acc: 0.6892    
Epoch 2/10
33364/33364 [=========================

33365/33365 [==============================] - 37s - loss: 0.6143 - acc: 0.7529    
Epoch 3/10
33365/33365 [==============================] - 37s - loss: 0.5798 - acc: 0.7665    
Epoch 4/10
33365/33365 [==============================] - 28s - loss: 0.5627 - acc: 0.7713    
Epoch 5/10
33365/33365 [==============================] - 28s - loss: 0.5511 - acc: 0.7765    
Epoch 6/10
33365/33365 [==============================] - 30s - loss: 0.5399 - acc: 0.7801    
Epoch 7/10
33365/33365 [==============================] - 30s - loss: 0.5338 - acc: 0.7837    
Epoch 8/10
33365/33365 [==============================] - 31s - loss: 0.5280 - acc: 0.7871    
Epoch 9/10
33365/33365 [==============================] - 31s - loss: 0.5218 - acc: 0.7871    
Epoch 10/10
33365/33365 [==============================] - 30s - loss: 0.7278 - acc: 0.6936    
Epoch 2/10
33365/33365 [==============================] - 32s - loss: 0.6103 - acc: 0.7553    
Epoch 3/10
33365/33365 [==============================] - 34